In [4]:
import pandas as pd
import numpy as np  
import re, json, logging , html, os , csv, codecs
from lib.config import connstr
from lib.context import context
from lib.utils import *  
from lib.script_generator import script_generator  
from lib.dbupdate_parser import dbupdate_parser   
ctx=context() 
ctx.logger.setLevel(logging.DEBUG) 
with open('config.json', 'r') as f: 
    ctx.config=json.loads(f.read())     
import warnings
warnings.filterwarnings('ignore')  
import string, random 
alphabet = string.ascii_letters + string.punctuation + string.whitespace
import datetime
from faker import Faker
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))
fake = Faker() 
def cve():
    return 'CVE-2024-' + str(fake.unique.random_int(min=10000, max=99999)) 
from datetime import datetime
today = datetime.today().strftime('%m-%d-%Y') 

dest=rf"C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Data\\"


In [15]:
result = json.load(codecs.open(r"C:\Users\timko\Downloads\ScubaResults_5df07c8c-3a87-4387.json", 'r', 'utf-8-sig'))
result['Raw'].keys()

dict_keys(['baseline_version', 'module_version', 'date', 'timestamp_zulu', 'report_uuid', 'tenant_details', 'scuba_config', 'conditional_access_policies', 'cap_table_data', 'authorization_policies', 'privileged_users', 'privileged_roles', 'service_plans', 'directory_settings', 'authentication_method', 'domain_settings', 'license_information', 'total_user_count', 'aad_successful_commands', 'aad_unsuccessful_commands', 'protection_policy_rules', 'atp_policy_rules', 'dlp_compliance_policies', 'dlp_compliance_rules', 'anti_phish_policies', 'protection_alerts', 'admin_audit_log_config', 'atp_policy_for_o365', 'total_users_without_advanced_audit', 'defender_license', 'defender_dlp_license', 'defender_successful_commands', 'defender_unsuccessful_commands', 'remote_domains', 'spf_records', 'dkim_config', 'dkim_records', 'dmarc_records', 'transport_config', 'sharing_policy', 'transport_rule', 'conn_filter', 'org_config', 'exo_successful_commands', 'exo_unsuccessful_commands', 'tenant_id', 'envi

In [ ]:

keys = list(result['Summary'].keys())
for k in keys: 
    del result['Summary'][k]
keys = list(result['Results'].keys())
for k in keys: 
    del result['Results'][k]
keys = [k for k in result['Raw'].keys() if 'scuba_config' not in k]
for k in keys: 
    del result['Raw'][k] 
 
json_string = json.dumps(result, indent=0)
with open(r"C:\Users\timko\Downloads\ScubaResults_valid.json", 'w') as f: 
    f.write(json_string)

{'MetaData': {'TenantId': '4617f433-5db6-42cc-879a-6b0e5bc5903c',
  'DisplayName': 'scubagcchigh',
  'DomainName': 'scubagcchigh.onmicrosoft.us',
  'ProductSuite': 'Microsoft 365',
  'ProductsAssessed': ['Azure Active Directory',
   'Microsoft 365 Defender',
   'Exchange Online',
   'Microsoft Power Platform',
   'SharePoint Online',
   'Microsoft Teams'],
  'ProductAbbreviationMapping': {'Microsoft 365 Defender': 'Defender',
   'SharePoint Online': 'SharePoint',
   'Azure Active Directory': 'AAD',
   'Microsoft Power Platform': 'PowerPlatform',
   'Microsoft Teams': 'Teams',
   'Exchange Online': 'EXO'},
  'Tool': 'ScubaGear',
  'ToolVersion': '1.4.0',
  'TimestampZulu': '2024-12-03T19:18:30.044Z',
  'ReportUUID': '5df07c8c-3a87-4387-a5f2-9d18ea00ab83'},
 'Results': {'AAD': [{'GroupName': 'Legacy Authentication',
    'GroupNumber': '1',
    'GroupReferenceURL': 'https://github.com/cisagov/ScubaGear/blob/v1.4.0/PowerShell/ScubaGear/baselines/aad.md#1-legacy-authentication',
    'Contro

In [ ]:
 
from faker.providers import BaseProvider

class CSProvider(BaseProvider): 
 
    def picklist(self, PK_PICKLISTTYPE):
        pk = self._all_picklists()
        DisplayValues = list(pk.loc[ pk["PK_PickListType"] ==int(PK_PICKLISTTYPE)  ]['DisplayValue'])  
        return self.random_elements(DisplayValues , 1 )[0]
    
    def component(self, pk_component = 0) : 
        df = self._all_components() 
        if pk_component > 0:
            df=df.loc[ df["AGENCY_PK"] == pk_component]
        lst = df.to_dict('records')  
        return self.random_elements(lst, 1 )[0]
    
    def agency(self) : 
        lst = list(self._all_components()['AGENCY'] )
        return self.random_elements(lst, 1 )[0]
      
    def _all_picklists(self): 
        return sql_todf("SELECT DisplayValue, PK_Picklist,PK_PickListType FROM vwPicklists  ", connstr)
    
    def _all_components(self):
        components = sql_todf(""" 
        SELECT A.PK_Component AGENCY_PK, A.Component AS AGENCY, A.Acronym AS AGENCY_Acronym, CONVERT(INT, b.PK_Component) AS Bureau_PK , b.Component AS Bureau, B.Acronym AS Bureau_Acronym 
        FROM [Component List] A 
        INNER JOIN [Component List] B ON B.FK_PK_Component=A.PK_Component AND B.IsActive = 1 
        WHERE A.IsActive = 1 AND A.FK_PK_Component IS NULL AND A.PK_Component < 200
        ORDER BY AGENCY_PK ASC
        """, connstr)
        components['Bureau_PK']=components['Bureau_PK'].fillna(0).astype(int) 
        return components
 

fake.add_provider(CSProvider)  
fake.component(24)

In [ ]:
df = df_fromDataFields(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\BODSCuBA\2024\2024_A_SCuBATenantInventory_1.aspx')
df

In [ ]:

lod=[]
for _ in range(0,5):
    d={}
    for i,r in df.iterrows(): 
        d[ r['HEADER'] ] = fake.md5()[:12]
        if int(r['PK_PICKLISTTYPE']) > 0: 
            d[ r['HEADER'] ] = get_randomDisplayValue( int(r['PK_PICKLISTTYPE']) ) 
        if 'notes' in r['HEADER']:
            d[ r['HEADER'] ] = fake.paragraph(nb_sentences=3)
        if 'Sub-agency' in r['HEADER']:
            d[ r['HEADER'] ] = fake.component(109)['Bureau']
    lod.append(d)
dff=pd.DataFrame(lod)
dff

In [ ]:
dff.to_excel(dest+'SCuBA_VALID.xlsx',   sheet_name='BOD 25-01 Inventory', index=False)

In [ ]:
lod = []
ab = 'B'
odds = 10
for i in range(1,1001):
    lod.append({
        'user_id':'u'+str(random.randint(100000,900000)),
        'timestamp': fake.date_time_this_year( ).strftime("%m-%d-%Y %H:%M:%S"),
        'page': ab,
        'converted' : (1 if random.randint(0,odds) < 5 else 0 ) 
    })
    if ab == 'B': 
        ab = 'A'
        odds=10
    else:   
        ab = 'B'
        odds=7

df=pd.DataFrame(lod)     
#  df.to_csv(fr'{ctx.get_download_path()}\ab_test_1000.csv', index=False)
#  df[:100].to_csv(fr'{ctx.get_download_path()}\ab_test_100.csv', index=False)
df.groupby('page').sum()

In [ ]:
 
fake.ipv6()
today
cve()
fake.paragraph(nb_sentences=3)
 




In [ ]:
# df = pd.read_excel(r'C:\Users\Tim\Downloads\FedRAMP Cloud Services - 060524.xlsx', sheet_name='FedRAMP Authorized Agency CSPs', header=0, usecols='A:Z' )  
df = pd.read_excel(r'C:\Users\Tim\Downloads\FedRAMP Authorized Agency CSPs - TEST.xlsx', sheet_name='FedRAMP Authorized Agency CSPs', header=0, usecols='A:Z' ) 

df = df.fillna('') 
df = df.rename(columns={'FedRAMP ID': 'FedRAMP Package ID'})
df['Agency'] = 'Department of Justice'
df['Bureau'] = '' 
df['FedRAMP ATO Issuance Date'] = '05/11/2023' 
df=pd.concat([df]*4, ignore_index=True)
df[:5000].to_excel(r'C:\Users\Tim\Downloads\fedramp_prepopulate_doj_5000.xlsx',   sheet_name='FedRAMP Authorized Agency CSPs', index=False)


In [ ]:
df[:1000].to_excel(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Data\fedramp_valid_10.xlsx',   sheet_name='CyberScope Sample Import', index=False)

In [ ]:

for i, row in df.iterrows():
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        s=f.read() 
    DATA_TYPE=row['DATA_TYPE']
    COLUMN_NAME=row['COLUMN_NAME']
    HEADER=row['HEADER']
    PK_PICKLISTTYPE=row['PK_PICKLISTTYPE']
    s=s.replace('{COLUMN_NAME}', COLUMN_NAME)
    if 'INT' in DATA_TYPE:  
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME+'_Display')
        s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{COLUMN_NAME}" DataValueField="YN" runat="server" />')  
    else:
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME )
        if 'Date' in COLUMN_NAME:  
            # print("const "+COLUMN_NAME+" = new CBDatePicker({ id: $(`[id$=_"+COLUMN_NAME+"]`).prop('id'), allowNa: true  }); ")  
            s=s.replace('<%-- --%>', f'<asp:HiddenField ID="{COLUMN_NAME}" runat="server" />') 
        else: 
            s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server" />') 
    #print(f"[{COLUMN_NAME}] [NVARCHAR](4000) NULL,")  
    print('{ field: "'+COLUMN_NAME+'", title: "'+HEADER+'" },')

In [ ]:
df[:4000].to_csv(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Data\ato_valid_4000.csv', index=False)

In [ ]:
df['Parent Agency'] = df['Parent Agency'].fillna('NULL')
df.iloc[16]['Parent Agency']

In [ ]:
df.to_excel(rf'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Data\ato_valid.xlsx',  sheet_name='ato_valid', index=False) 
df

In [ ]:
for c in df.columns:
    s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if "Date" in c:
        s = f'<CB:DataField  DBColumnName="{c.replace(" ","")}" Require="False" ImportColumnName="{c}"  runat="server"/>  '
    print(s)
 

In [ ]:
print(ctx.get_download_path())
file=rf'{ctx.get_download_path()}\ATO.csv'  
df=pd.read_excel(file, sheet_name='Cost & Funding Template', header=5, usecols='B:Z' )    
df.columns

In [ ]:
df.columns = list(map(lambda c: '' + c.replace(' ', ''), df.columns))

In [ ]:
s = SQL_INSERT_FROM_DF(df)[1]
print(s)

In [ ]:
records=20000
dff=pd.concat([df]*records, ignore_index=True)
dff['Inventory Entry Identifier'] = dff['Inventory Entry Identifier'].apply(lambda s: id_generator(10))
dff['Applicable Cost Categories'] = dff['Applicable Cost Categories'].apply(lambda s: rand_pick('SUBSTAT', 'DisplayValue'))
dff['Applicable Technology Categories'] = dff['Applicable Technology Categories'].apply(lambda s: rand_pick('IDPS', 'DisplayValue'))

dff.to_excel(fr'C:\Users\Tim\Downloads\CF_{records}.xlsx',  sheet_name='Cost & Funding Template', index=False) 



In [ ]:
file=f'{ctx.get_down_dir()}MFA.xlsx'  
df=pd.read_excel(file, sheet_name=int(1), header=0, usecols='A:Z' )   

In [ ]:
dff=pd.concat([df]*50, ignore_index=True)
dff['System ID (Optional)'] = dff['System ID (Optional)'].apply(lambda s: id_generator(10))
dff['High Value Asset ID (Optional)'] = dff['High Value Asset ID (Optional)'].apply(lambda s: id_generator(10))
dff['System Name'] = dff['System Name'].apply(lambda s: id_generator(10))
dff['System Description'] = dff['System Description'].apply(lambda s: fake.paragraph(nb_sentences=1))
dff.head()

In [ ]:
dff.to_excel(f'{ctx.get_down_dir()}MFA_50.xlsx',  sheet_name='Sheet2', index=False) 

In [ ]:
sql=""" SELECT COLUMN_NAME, DATA_TYPE  FROM INFORMATION_SCHEMA.COLUMNS
 WHERE TABLE_NAME LIKE '%MFA%' AND (ORDINAL_POSITION > 0  ) ORDER BY ORDINAL_POSITION ASC """
df=sql_todf(sql, connstr)   
df